In [865]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
import numpy as np

In [941]:
# Declare empty list to append dictionaries to
data_list = []

In [942]:
# Declare data sources
url1 = "https://redplanetscience.com"
url2 = "https://galaxyfacts-mars.com/"
url3 = "https://marshemispheres.com/"
url4 = "https://spaceimages-mars.com/"

In [943]:
# Make a table full of facts from earth and mars
tables = pd.read_html(url2)
df = tables[0]

In [944]:
# df_dict = df.to_dict('records')
df_dict = {"category":(df[0].to_list()),"mars":(df[1].to_list()),"earth":(df[2].to_list())}

In [945]:
# Add the newly scraped table to 
data_list.append(df_dict)

In [946]:
# Make a path for browser to visit and scrape data from latest news article
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', ** executable_path, headless=False)
browser.visit(url1)

In [947]:
# Use beautiful soup to parse html
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [948]:
# Get the newest title
title = soup.find('div', class_='content_title')
title = title.text.strip()

In [949]:
# Get the newest paragraph
paragraph = soup.find('div', class_='article_teaser_body')
paragraph = paragraph.text.strip()

In [950]:
# Store latest article into a dictionary
latest_news = {
    "title": title,
    "paragraph": paragraph
}

In [951]:
# Append dictionary to list
data_list.append(latest_news)

In [952]:
# Make a path for browser to turn into soup and scrape the featured img
browser.visit(url4)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [953]:
# Click on the full image button
browser.links.find_by_partial_text('FULL IMAGE').click()

In [954]:
# Select the tag with the img link and parse it into a string link.
img = soup.find('a', class_='showimg fancybox-thumbs')
img = str(img)
img = "https://spaceimages-mars.com/"+img.split('"')[3]

In [955]:
# Store the link into a dictionary
featured_img = {
    "featured_img": img
}

In [956]:
# Append dictionary to list
data_list.append(featured_img)

In [957]:
# Make a path for browser to turn into soup and scrape the hemisphere titles and images
browser.visit(url3)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
item = soup.find_all('div', class_='item')

In [958]:
# Declare empty arrays to add links of images and titles
title_list = []
img_list = []

In [959]:
# Loop through and add titles and links to the empty array
for pic in item:
    header = ((str(pic.h3)).split("<h3>")[1])
    header = (header.split("</h3>")[0])
    title_list.append(header)
    img = "https://marshemispheres.com/"+str(pic).split('"')[11]
    img_list.append(img)

In [961]:
# Store the links and titles into a dictionary
hemispheres = {
    "titles":title_list,
    "images":img_list
}

In [962]:
# Append dictionary to list
data_list.append(hemispheres)

In [963]:
# Close browser
browser.quit()

In [918]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [919]:
#Creates/Defines database in Mongo
db = client.mars_db

In [920]:
#Define tables in database
mars = db.mars

In [921]:
#Creates and inserts all dataframes into their tables
mars.insert_many(data_list)